# 코드 시작 전 설명
1. 음성 데이터에 대한 이론 및 과정을 잘 모르시는 분은 코드를 작성하기 전 overview를 한번 정독하시는 것을 권합니다.

2. 스켈레톤 코드가 적혀있는 해당 ipynb 파일을 다운받아 자신의 작업공간(코랩, 캐글노트북 등등)에서 코드를 작성합니다.

3. 각 코드에는 빈칸과 함께 구현 가이드 라인이 제공됩니다. 해당 가이드라인을 따라 코드를 작성하여 베이스 라인 성능을 달성합니다.
   진행에 어려움이 있으신 경우에는 feature 가공에 사용되는 함수의 documentation 설명을 읽으시는 것을 권합니다.
   
4. (선택)베이스라인에 도달하였다면, 음성 feature를 새롭게 가공하여 추가적인 성능 향상을 달성해봅니다.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp-p6/train_data.csv
/kaggle/input/2021-ml-tp-p6/test_data.csv
/kaggle/input/2021-ml-tp-p6/sample_submit.csv
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_132.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_185.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_268.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_344.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_093.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_131.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_327.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_120.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_064.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_371.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_028.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_250.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_408.wav
/kaggle/input/2021-ml-tp-p6/test_data/test_data/test_334.wav
/kaggle/input/2

In [2]:
import pandas as pd
import numpy as np
import sklearn
import os
from os.path import join

In [3]:
#DATA Load

DATA_PATH = join('/kaggle','input','2021-ml-tp-p6')
pd_train = pd.read_csv(join(DATA_PATH, 'train_data.csv'))
pd_test = pd.read_csv(join(DATA_PATH, 'test_data.csv'))

print(pd_train.info(), pd_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         1008 non-null   int64 
 1   file_name  1008 non-null   object
 2   emotion    1008 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         432 non-null    int64 
 1   file_name  432 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.9+ KB
None None


# **Feature 추출**

* extract_feature 함수는 음성 데이터를 읽어서 다음의 과정들을 거치며 feature를 추출합니다.(함수(입력) --> 출력)
 1. Sampling&Quantization(continuos audio signal) --> Discrete audio signal
 2. Short Time Fourier Transfrom(Discrete audio signal) --> Spectrogram
 3. Mel-Filter(Spectrogram) --> Mel-Spectrogram
 4. Discrete Cosine Transform(Mel-Spectrogram) --> Mel Frequency Cepstrum Coefficient
 
위에 과정들은 모두 librosa 라이브러리에서 제공하는 함수들을 통하여 구현가능하며 사용되는 함수는 다음과 같습니다.

1. librosa.load : Continuos audio signal(.wav파일)을 Discrete audio signal로 읽음. 
2. librosa.stft : Discrte audio signal을 windowing하여 프레임별로 나눈 후 FFT(Fast Fourier Transform)을 수행
[stft_documentation](https://librosa.org/doc/latest/generated/librosa.stft.html?highlight=stft)
3. librosa.feature.melspectrogram : stft 함수를 통하여 구한 spectrogram을 입력으로 Mel-filter를 적용함으로써 Mel-Spectrogram 생성[melspectrogram_documentation](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html?highlight=melspectrogram)
4. librosa.feature.mfcc : melspectrogram 함수를 통하여 구한 Mel-spectrogram을 입력으로 DCT를 수행함으로써 MFCC를 생성 [mfcc_documentation](https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html?highlight=mfcc)

In [4]:
import librosa
import glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import librosa, librosa.display 


# -------------------------------------
# extract_feature(file_name): <= 코드를 추가하여 음성 feature를 추출하는 코드를 완성하세요
# -------------------------------------
# 목적: MFCC를 비롯한 여러 음성 feature를 추출
# 입력인자: .wav 음성 파일의 이름
# 출력인자: 입력 파일을 가공한 feature들 (Spectrogram, Mel-spectrogram, MFCC)
# -------------------------------------


def extract_feature(file_name):
    
    
    result=np.array([])
    X, sample_rate = librosa.load(file_name, sr=22050)
    #----------step3. spectrogram을 구하세요.---------------------
    # 구현 가이드 라인(3)
    # ------------------------------------------------------------------------------
    # 1. 입력 신호(X)를 librosa.stft 함수의 입력으로 하여 spectrogram을 구하세요.
    #   -참고) 사람의 음성은 20~40(ms) 사이 시간 내에서 현재 말하는 발음을 변경할 수 없다고 합니다.
    #       시간축에 대한 구간을 나눌 때 20~40(ms) 이내 간격으로 나누기 위하여 n_fft 파라미터 값을 조정해주세요.(베이스라인 성능은 23ms 간격으로 나누었습니다.)
    #       정확한 조정을 위하여 librosa documentation에서 librosa.stft 함수 내 n_fft 파라미터 설명을 참조하세요.
    #
    # 2. spectrogram에 절대값을 취하여 복소수 형태의 값을 바꾸세요.
    #
    # 3. 구한 spectrogram을 학습에 사용하기 위하여 프레임 축의 평균값을 취한 뒤 spectrogram_feature에 저장해주세요
    #   -참고) spectrogram의 shape은 (frequency의 길이, 프레임 수)로 이루어져있습니다.
    #       (np.mean함수를 사용하여 spectrogram의 shape이 (1, frequency의 길이)가 되면 성공.)
    # -----------------------------------------------------------------------------
    #librosa 문서를 확인해보니 입력신호와 n_fft 인자가 있는데, 일반적으로 사람 목소리를 할 경우 512로 n_fft를 잡으라 하여 설정함.
    #그 외 나머지는 가이드라인에 따라 작성함
    spectrogram = librosa.stft(y=X, n_fft=512)
    spectrogram = np.abs(spectrogram)
    spectrogram_feature = np.mean(spectrogram, axis=1)
    #-------------------------------------------------------------------------------
    
    #----------step4. Mel-spectrogram을 구하세요.---------------------
    # 구현 가이드 라인(4)
    # ------------------------------------------------------------------------------
    # 1. step3-2에서 구한 spectrogram을 제곱하여 power spectrogram을 만드세요.
    #
    # 2. power spectrogram을 librosa.feature.melspectrogram 함수의 입력으로 하여 mel-spectrogram을 구하세요.
    #   - 참고) documentation을 통해 librosa.feature.melspectrogram 함수의 입력 인자를 꼭 확인하셔서 올바르게 넣어주세요.
    #
    # 3. step4-2에서 구한 mel-spectrogram은 power-magnitude 값입니다. librosa.power_to_db함수를 통하여 power magnitude를 데쉬벨(db)로 변환하세요.
    #
    # 4. 구한 mel-spectrogram을 학습에 사용하기 위하여 프레임 축의 평균값을 취한 뒤 mel_spectrogram_feature에 저장해주세요
    #   -참고) mel-spectrogram의 shape은 (mel filter의 길이, 프레임 수)로 이루어져있습니다.
    #       (np.mean함수를 사용하여 mel-spectrogram의 shape이 (1, mel filter의 길이)가 되면 성공.)
    # -----------------------------------------------------------------------------
    #일단 np.power함수를 통해 제곱값을 저장.
    #그 후 librosa의 melspectrogram 함수를 보니 sr, S, n_fft가 있기에 sr은 위와 같이 22050
    #스펙트로그램은 제곱한 값을, n_fft 값은 위와 동일하게 설정하였다.
    #그 외 부분은 가이드라인을 따라 작성하였다.
    power_spectrogram = np.power(spectrogram, 2)
    mel_spectrogram = librosa.feature.melspectrogram(sr=22050,S=power_spectrogram, n_fft=512)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    mel_spectrogram_feature= np.mean(mel_spectrogram, axis=1)
    #-------------------------------------------------------------------------------

    #----------step5. MFCC를 구하세요.---------------------
    # 구현 가이드 라인(5)
    # ------------------------------------------------------------------------------ 
    # 1. step4-3에서 데쉬벨로 변환한 mel-spectrogram을 librosa.feature.mfcc 함수의 입력으로 하여 MFCC를 구하세요.
    #   - 참고) documentation을 통해 librosa.feature.mfcc 함수의 입력 인자를 꼭 확인하셔서 올바르게 넣어주세요.
    #
    # 2. 구한 MFCC 학습에 사용하기 위하여 프레임 축의 평균값을 취한 뒤 mfcc_feature에 저장해주세요
    # -참고) MFCC shape은 (MFCC의 길이, 프레임 수)로 이루어져있습니다.
    #       (np.mean함수를 사용하여 MFCC의 shape이 (1, MFCC의 길이)가 되면 성공.)
    # -----------------------------------------------------------------------------
    #librosa 문서를 보니 인자로 sr과 스펙트로그램을 넣기에 22050과 mel_spectrogram으로 설정하고 나머지는 가이드라인에 맞추었다.
    mfcc = librosa.feature.mfcc(sr = 22050, S = mel_spectrogram)
    mfcc_feature = np.mean(mfcc, axis=1)
    #-------------------------------------------------------------------------------

    return spectrogram_feature, mel_spectrogram_feature, mfcc_feature 

# **데이터 불러오기**

### csv파일에 저장된 파일 이름과 학습용 label을 로드하여 학습 및 평가용 데이터를 불러오세요.

In [5]:
#DataFlair - Load the data and extract features for each sound file
from tqdm import tqdm
def load_data(data_info, isTrain=True):
    
    PATH = join('/kaggle','input','2021-ml-tp-p6')
    if isTrain:
        train_data = {'spectrogram':[],'mel':[],'mfcc':[]}#음성 feature들을 담는 dictionary
        train_label = []#학습에 사용할 label을 담는 list
        
        file_list = data_info['file_name']
        emotion_list = data_info['emotion']
        for file_name, emotion in tqdm(zip(file_list, emotion_list)):
            # ------------- step1. 학습용 데이터로더 코드를 작성하세요.----------------------
            # 구현 가이드라인  (1)
            # ------------------------------------------------------------
            # train.csv 파일에 있는 음성 파일의 이름과 emotion 정보를 통하여 학습용 데이터를 로드하세요.
            # 음성 파일의 정확한 경로 명을 extract_feature 함수의 입력으로 넣을 수 있게 경로를 잘 설정해보세요.
            # extract_feature를 통해 구한 음성 feature들을 train_data 사전 속 배열에 알맞게 append 해주세요. ex) train_data['spectrogram'].append(spectrogram_feature)
            # ------------------------------------------------------------
            # 구현 가이드라인을 참고하여 코드를 작성해보세요.
            # 결과값인 감정을 label에 추가한 후 위에서 추가한 데이터 경로를 확인하여 경로를 만들었다.
            # 각 스펙트로그램의 특징을 함수를 이용하여 받아오고 train_data에 추가해주었다.
            train_label.append(emotion)
            path = join(PATH, 'train_data/train_data', file_name)
            spectrogram_feature, mel_spectrogram_feature, mfcc_feature = extract_feature(path)
            train_data['spectrogram'].append(spectrogram_feature)
            train_data['mel'].append(mel_spectrogram_feature)
            train_data['mfcc'].append(mfcc_feature)
            
            #----------------------------------------------------------------------------------------- 
            
        return train_data, np.array(train_label)
    
    else:
        test_data = {'spectrogram':[],'mel':[],'mfcc':[]}#음성 feature들을 담는 dictionary
        file_list = data_info['file_name']
    
        for file_name in tqdm(file_list):
            # -------------step2. 평가용 데이터로더 코드를 작성하세요.-----------------
            # 구현 가이드라인  (2)   
            # ------------------------------------------------------------
            # test.csv 파일에 있는 음성 파일의 이름정보를 통하여 평가용 데이터를 로드하세요.
            # 음성 파일의 정확한 경로 명을 extract_feature 함수의 입력으로 넣을 수 있게 경로를 잘 설정해보세요.
            # extract_feature를 통해 구한 음성 feature들을 test_data 사전 속 배열에 알맞게 append 해주세요. ex) test_data['spectrogram'].append(spectrogram_feature)
            # ------------------------------------------------------------
            # 구현 가이드라인을 참고하여 코드를 작성해보세요.
            # test에서는 결과값인 emotion이 존재하지 않기에 emotion 파트를 제외하고는 위 train_data와 동일하게 구성하였다.
            path = join(PATH, 'test_data/test_data', file_name)
            spectrogram_feature, mel_spectrogram_feature, mfcc_feature = extract_feature(path)
            test_data['spectrogram'].append(spectrogram_feature)
            test_data['mel'].append(mel_spectrogram_feature)
            test_data['mfcc'].append(mfcc_feature)
            
            #----------------------------------------------------------------------------------------- 
            
        return test_data

#DataFlair - Split the dataset
train_data, y_train = load_data(pd_train)
test_data = load_data(pd_test, isTrain=False)

1008it [02:46,  6.06it/s]
100%|██████████| 432/432 [01:10<00:00,  6.09it/s]


# 모델 학습 및 추론
위에서 우리는 
1. 음성 신호에 대하여 Short Time Fourier Transform(stft)를 통해 **spectrogram**을 구하고,
2. 구한 spectrogram에 mel-filter를 씌워 **mel-spectrogram**을 구했으며,
3. 마지막으로 mel-spectrogram에 Discrete cosine transform(DCT) 취해줌으로써 **mfcc**까지 계산하였습니다.

위에서 구한 feature들은 모두 음성 데이터를 활용하는 다양한 작업(사람 음성 분류, 음성을 통한 감정 분류 등)에서 모델 학습에 사용할 수 있는 feature들입니다.
각각의 **feature들에 따른 모델의 정확도가 얼마나 차이**나는지를 확인해봅시다.

- 베이스라인의 분류기는 **RandomForestClassifier**를 사용합니다.
- **random_state에 따른 성능 차이가 발생하오니 반드시 random_state를 1로 고정해주시길 바랍니다.**

In [6]:
#AutoML 적용해보기
!apt-get remove swig
!apt-get install swig3.0 build-essential -y

!ln -s /usr/ing/swig3.0 /usr/bin/swig

!apt-get install build-essential

!pip install --upgrade setuptools

!pip install auto-sklearn

import autosklearn.classification




Package 'swig' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.



build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig3.0
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 1094 kB of archives.
After this operation, 5499 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1094 kB]
Fetched 1094 kB in 2s (507 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package swig3.0.
(Reading database ... 95165 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Setting up swig3.0 (3.0.12-1) ...



build-essential is already the newest version (12.4ubuntu1).
0 upgraded, 0 newly installed, 0 to remo

/opt/conda/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [7]:
#RandomForestClassifier로 음성 감정 분류 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

#앙상블 - 투표, 배깅, 아다부스트 방식 적용해보기
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#그리드 서치
from sklearn.model_selection import GridSearchCV

sample = pd.read_csv(join(DATA_PATH,'sample_submit.csv'))

for feature_name in train_data.keys():
    # ------------- step6. 음성 feature들을 가지고 모델을 학습하세요.----------------------
    # 구현 가이드라인  (6)
    # ------------------------------------------------------------
    # dictionary 형태의 train_data 변수 내에는 spectrogram, mel-spectrogram, mfcc feature들이 존재합니다.
    # 반복문을 통해 각 종류의 feature를 하나씩 불러오세요. ex) x_train = np.array(train_data[feature_name])
    # 불러온 feature로 모델을 학습 및 추론 후 sample submit파일에 저장하세요.
    # ------------------------------------------------------------
    # 구현 가이드라인을 참고하여 코드를 작성해보세요.
    
    # np.array를 이용해 각 train 데이터를 저장했다.
    x_train = np.array(train_data[feature_name])
    x_test = np.array(test_data[feature_name])
    
    ###########################################################
    #아래 GridSearch로 찾은 파라미터를 단일 랜덤 포레스트 분류기
    #해당 파트를 사용할 경우 성능 0.60648
    
    rfc = RandomForestClassifier(random_state=1, criterion='entropy', n_estimators=200)
    #찾은 최적 파라미터를 통한 학습 및 예측
    rfc.fit(x_train, y_train)
    predict = rfc.predict(x_test)
    
    ###########################################################
    
    ###########################################################
    #그리드 서치를 이용해서 최적의 파라미터를 찾아보았다.
    #entropy와 n_estimator 200에서 최고점이 나오는것을 확인했다.
    #max_depth도 조정해보았으나 19미만에서는 점수가 떨어지는것을 확인하였다.
    #class_weight를 넣었다가 성능이 떨어지는 것을 확인하여 제외하였다.
    '''
    #params = {criterion:['entropy', 'gini'], 'n_estimators':[150, 200, 250, 300]}
    #clf = GridSearchCV(rfc, params, cv=10, n_jobs=-1)
    #clf.fit(x_train, y_train)
    #predict = clf.predict(x_test)
    #print(clf.best_params_)
    '''
    ###########################################################
    
    ###########################################################
    #AutoML 테스트
    #해당 파트를 사용할 경우 성능이 가장 높게 나왔었으나,
    #실행시마다 성능이 다르게 나와 제출할 수 없었습니다.
    #아래 autoML실행시 사용됐던 가장 높은 파라미터를 주석으로 추가해두었습니다.
    '''
    cls = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=120,
        include_estimators=[
            'k_nearest_neighbors',
            'random_forest',
            'lda',
            'qda',
            'decision_tree',
            'libsvm_svc'
        ],
        n_jobs=-1,
        seed=1
    )
    
    #autoML 학습 및 예측
    cls.fit(x_train, y_train)
    predict = cls.predict(x_test)
    '''
    ###########################################################
    
    ###########################################################
    '''
    #투표 방식 테스트
    knn = KNeighborsClassifier(n_neighbors=3)
    svm = SVC(class_weight='balanced', probability=True)
    voting_estimators = [('rfc', rfc), ('knn', knn), ('svm', svm)]
    voting = VotingClassifier(estimators = voting_estimators, voting='soft')
    
    params = {
        'svm__C' : [3000, 6000, 10000]
    }
    
    clf = GridSearchCV(voting, params, cv=10)
    
    clf.fit(x_train, y_train)
    
    print(clf.best_params_)
    predict = clf.predict(x_test)
    '''
    ###########################################################
    
    ###########################################################
    #Bagging 테스트
    '''
    bagging = BaggingClassifier(base_estimator = rfc,
                               n_estimators=200,
                               max_samples=1.0,
                               max_features=1.0,
                               bootstrap=True,
                               bootstrap_features=False,
                               n_jobs=-1,
                               random_state=1
                               )
    
    bagging.fit(x_train, y_train)
    predict = bagging.predict(x_test)
    '''
    ###########################################################
    
    ###########################################################
    #AdaBoost 테스트
    '''
    adaboost = AdaBoostClassifier(base_estimator = rfc,
                                 n_estimators=500,
                                 learning_rate=0.1,
                                 random_state=1)
    
    adaboost.fit(x_train, y_train)
    predict = adaboost.predict(x_test)
    '''
    ###########################################################
    
    
    #Sample submit file 저장
    sample['emotion'] = predict.reshape(-1,1)
    sample.to_csv(join(feature_name+'.csv'),index=False,header=True)

# 결론
#autoML 사용하지 않았을시 일반 랜덤포레스트 : 0.60648
#다양한 분류기를 포함한 autoML 사용했을시 : 0.62962
#bagging의 n_estimator 갯수가 높아짐에 따라 메모리 문제로 실행이 되지 않아 
#낮춘결과 성능이 그렇게 좋게 나오지는 않았다.

In [8]:
#AutoML에서 제일 높은 성적으로 나왔을때의 파라미터
'''
#autoML 사용시 사용된 파라미터(분류기, 분류기에 사용된 파라미터 등등)
balancing:strategy: weighting
classifier:__choice__: lda
data_preprocessing:categorical_transformer:categorical_encoding:__choice__: one_hot_encoding
data_preprocessing:categorical_transformer:category_coalescence:__choice__: minority_coalescer
data_preprocessing:numerical_transformer:imputation:strategy: most_frequent
data_preprocessing:numerical_transformer:rescaling:__choice__: minmax
feature_preprocessor:__choice__: kitchen_sinks
classifier:lda:shrinkage: auto
classifier:lda:tol: 0.00045007567068269274
data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction: 0.0074657344346488255
feature_preprocessor:kitchen_sinks:gamma: 2.0209303149330458
feature_preprocessor:kitchen_sinks:n_components: 3790
'''
#AutoML에서 test_scores가 1등으로 나와 사용된 파라미터를 알아보기 위한 로직
#1등인 랭크의 인덱스를 뽑은 후
rank_index = np.where(cls.cv_results_['rank_test_scores']==1)[0][0]
print(rank_index)

#해당 인덱스의 값을 순차적으로 뽑아준다.
for key in cls.cv_results_['params'][rank_index]:
    print(key + ":", end=' ')
    print(cls.cv_results_['params'][rank_index][key])

NameError: name 'cls' is not defined